In [2]:
import gradio as gr
import requests
import json
import sqlite3
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
from typing import List, Dict, Optional, Tuple
import random
import re
import hashlib
from dataclasses import dataclass
from collections import defaultdict, Counter
import threading
import time

@dataclass
class RestaurantProfile:
    cuisine_type: str
    style: str
    location: str
    target_audience: str
    budget_range: str
    theme: str
    atmosphere: str
    unique_features: str
    competitor_analysis: str

class AdvancedRestaurantGenerator:
    def __init__(self, api_key: str):
        self.api_key = "YOU API KEYS"
        self.base_url = "https://aimlapi.com/app/"   #you can use this website for free API keys
        self.db_path = "restaurant_generator.db"
        self.init_database()

        # Advanced name patterns and components
        self.name_patterns = {
            "classic": ["{adjective} {cuisine_word}", "{location} {type}", "The {concept}"],
            "modern": ["{fusion_word}{type}", "{trendy_prefix}.{suffix}", "{concept} + {style}"],
            "playful": ["{pun_word} {type}", "{alliteration}", "{rhyme_word}"],
            "elegant": ["{french_article} {elegant_word}", "{descriptor} {cuisine_word}", "Maison {concept}"],
            "local": ["{city_name} {type}", "{neighborhood} {style}", "{landmark} {cuisine}"]
        }

        self.linguistic_components = {
            "prefixes": ["Bella", "Casa", "Chez", "La", "Le", "Il", "Der", "Das", "El"],
            "suffixes": ["ria", "etto", "ina", "ito", "haus", "hof", "teca", "bar"],
            "fusion_connectors": ["&", "+", "x", "meets", "and", "with"],
            "modern_elements": ["Co.", "Lab", "Studio", "Works", "Collective", "House"]
        }

    def init_database(self):
        """Initialize SQLite database for storing generation history and analytics"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        # Main generations table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS generations (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
                cuisine_type TEXT,
                style TEXT,
                location TEXT,
                target_audience TEXT,
                budget_range TEXT,
                theme TEXT,
                atmosphere TEXT,
                unique_features TEXT,
                competitor_analysis TEXT,
                generated_names TEXT,
                selected_name TEXT,
                user_rating INTEGER,
                session_id TEXT
            )
        ''')

        # Analytics table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS analytics (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                date DATE,
                total_generations INTEGER,
                popular_cuisine TEXT,
                popular_style TEXT,
                avg_rating REAL
            )
        ''')

        # Name components frequency table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS name_components (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                component TEXT,
                component_type TEXT,
                frequency INTEGER,
                success_rate REAL
            )
        ''')

        conn.commit()
        conn.close()

    def generate_advanced_names(self, profile: RestaurantProfile, count: int = 8) -> Dict:
        """Generate restaurant names using advanced AI prompting and multiple strategies"""

        # Create comprehensive prompt with context
        system_prompt = """You are an expert restaurant naming consultant with 20+ years of experience in branding and marketing. You understand cultural nuances, linguistic patterns, and market positioning."""

        user_prompt = f"""Create {count} restaurant names for this detailed profile:

RESTAURANT PROFILE:
- Cuisine: {profile.cuisine_type}
- Style: {profile.style}
- Location: {profile.location}
- Target Audience: {profile.target_audience}
- Budget Range: {profile.budget_range}
- Theme: {profile.theme}
- Atmosphere: {profile.atmosphere}
- Unique Features: {profile.unique_features}
- Competitor Analysis: {profile.competitor_analysis}

NAMING REQUIREMENTS:
1. Create names that reflect the cuisine and atmosphere
2. Consider the target audience and location
3. Ensure names are memorable and pronounceable
4. Avoid trademark conflicts with major chains
5. Include cultural authenticity where appropriate
6. Consider SEO and social media friendliness

DELIVERY FORMAT:
For each name, provide:
- Name
- Reasoning (2-3 sentences explaining the choice)
- Market positioning (upscale/casual/trendy/traditional)
- Memorable factor (1-10)

Please provide exactly {count} names in this JSON format:
{{"names": [{{"name": "Restaurant Name", "reasoning": "Why this name works...", "positioning": "market position", "memorable_score": 8}}]}}"""

        try:
            # Primary AI generation
            ai_names = self._call_ai_api(system_prompt, user_prompt)

            # Fallback algorithmic generation
            algorithmic_names = self._generate_algorithmic_names(profile, count//2)

            # Combine and enhance results
            all_names = ai_names + algorithmic_names

            # Apply linguistic analysis and scoring
            scored_names = self._analyze_and_score_names(all_names, profile)

            # Store in database
            self._store_generation(profile, scored_names)

            return {
                "names": scored_names[:count],
                "generation_stats": {
                    "total_generated": len(all_names),
                    "ai_generated": len(ai_names),
                    "algorithmic_generated": len(algorithmic_names),
                    "average_score": sum(name.get("score", 0) for name in scored_names) / len(scored_names) if scored_names else 0
                }
            }

        except Exception as e:
            print(f"Generation error: {e}")
            return {"names": self._generate_fallback_names(profile, count), "error": str(e)}

    def _call_ai_api(self, system_prompt: str, user_prompt: str) -> List[Dict]:
        """Enhanced AI API call with better error handling"""
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }

        data = {
            "model": "meta-llama/Llama-3.2-3B-Instruct-Turbo",
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            "max_tokens": 800,
            "temperature": 0.8,
            "top_p": 0.9,
            "repetition_penalty": 1.1
        }

        response = requests.post(self.base_url, headers=headers, json=data, timeout=45)

        if response.status_code == 200:
            result = response.json()
            content = result['choices'][0]['message']['content'].strip()

            # Try to parse JSON response
            try:
                parsed = json.loads(content)
                return parsed.get("names", [])
            except json.JSONDecodeError:
                # Fallback parsing for non-JSON responses
                lines = [line.strip() for line in content.split('\n') if line.strip()]
                names = []
                for line in lines[:8]:
                    if line and not line.startswith('#'):
                        name = re.sub(r'^\d+\.?\s*', '', line)  # Remove numbering
                        names.append({
                            "name": name,
                            "reasoning": "AI generated name",
                            "positioning": "casual",
                            "memorable_score": 7
                        })
                return names
        else:
            raise Exception(f"API call failed: {response.status_code}")

    def _generate_algorithmic_names(self, profile: RestaurantProfile, count: int) -> List[Dict]:
        """Advanced algorithmic name generation using linguistic patterns"""
        names = []

        # Cuisine-specific word banks
        cuisine_banks = {
            "Italian": {"words": ["Bella", "Casa", "Nonna", "Palazzo", "Villa", "Tavola", "Amore", "Sole"], "endings": ["ria", "etto", "ina", "ito"]},
            "French": {"words": ["Petit", "Grand", "Chez", "Maison", "Bistro", "Cafe", "Bon", "Belle"], "endings": ["erie", "oux", "eau", "ette"]},
            "Japanese": {"words": ["Sakura", "Yuki", "Hana", "Kaze", "Mizu", "Tsuki", "Aki", "Natsu"], "endings": ["ya", "tei", "sushi", "ramen"]},
            "Mexican": {"words": ["Casa", "Cantina", "Fiesta", "Sol", "Luna", "Mesa", "Pueblo", "Corazon"], "endings": ["ita", "ito", "eria", "ado"]},
            "Indian": {"words": ["Maharaja", "Taj", "Spice", "Curry", "Masala", "Palace", "Garden", "Namaste"], "endings": ["ana", "wala", "ghar", "mahal"]},
            "Chinese": {"words": ["Golden", "Dragon", "Phoenix", "Jade", "Pearl", "Bamboo", "Lotus", "Garden"], "endings": ["house", "garden", "palace", "dynasty"]}
        }

        # Style-based modifiers
        style_modifiers = {
            "Fine Dining": ["Royal", "Grand", "Premier", "Elite", "Signature", "Prestige"],
            "Casual": ["Corner", "Family", "Neighborhood", "Local", "Friendly", "Cozy"],
            "Fast Casual": ["Quick", "Express", "Fresh", "Urban", "Modern", "Grab"],
            "Trendy": ["Hip", "Cool", "Fresh", "Urban", "Modern", "New"],
            "Traditional": ["Old", "Classic", "Heritage", "Authentic", "Original", "Time-honored"]
        }

        bank = cuisine_banks.get(profile.cuisine_type, cuisine_banks["Italian"])
        modifiers = style_modifiers.get(profile.style, style_modifiers["Casual"])

        # Generate names using different patterns
        for i in range(count):
            pattern_type = random.choice(list(self.name_patterns.keys()))

            if pattern_type == "classic":
                name = f"{random.choice(modifiers)} {random.choice(bank['words'])} {random.choice(['Kitchen', 'Restaurant', 'Bistro', 'Cafe'])}"
            elif pattern_type == "modern":
                base = random.choice(bank['words'])
                ending = random.choice(bank['endings'])
                name = f"{base}{ending} {random.choice(['Co.', 'Lab', 'House', 'Works'])}"
            elif pattern_type == "elegant":
                article = random.choice(["Le", "La", "Il", "Der", "El"])
                name = f"{article} {random.choice(bank['words'])} {random.choice(bank['endings'])}"
            else:
                # Fusion approach
                word1 = random.choice(bank['words'])
                word2 = random.choice(modifiers)
                connector = random.choice([" & ", " + ", " x ", ""])
                name = f"{word1}{connector}{word2}"

            names.append({
                "name": name,
                "reasoning": f"Algorithmic generation using {pattern_type} pattern",
                "positioning": profile.style.lower(),
                "memorable_score": random.randint(6, 9),
                "source": "algorithmic"
            })

        return names

    def _analyze_and_score_names(self, names: List[Dict], profile: RestaurantProfile) -> List[Dict]:
        """Advanced linguistic analysis and scoring"""
        scored_names = []

        for name_data in names:
            name = name_data.get("name", "")
            if not name:
                continue

            score = 0
            analysis = {}

            # Length scoring (8-20 characters optimal)
            length = len(name)
            if 8 <= length <= 20:
                score += 2
            elif 6 <= length <= 25:
                score += 1

            analysis["length_score"] = min(2, max(0, 2 - abs(length - 14) / 7))

            # Pronounceability (vowel/consonant ratio)
            vowels = len(re.findall(r'[aeiouAEIOU]', name))
            consonants = len(re.findall(r'[bcdfghjklmnpqrstvwxyzBCDFGHJKLMNPQRSTVWXYZ]', name))
            if consonants > 0:
                ratio = vowels / consonants
                if 0.3 <= ratio <= 0.8:
                    score += 2
                analysis["pronounce_score"] = min(2, max(0, 2 - abs(ratio - 0.55) * 4))

            # Memorability factors
            # Alliteration
            words = name.split()
            if len(words) >= 2 and words[0][0].lower() == words[1][0].lower():
                score += 1
                analysis["alliteration"] = True

            # Uniqueness (avoid common words)
            common_words = {"restaurant", "cafe", "bistro", "kitchen", "grill", "bar", "house"}
            unique_words = [w.lower() for w in words if w.lower() not in common_words]
            if len(unique_words) >= len(words) * 0.7:
                score += 1
                analysis["uniqueness_score"] = len(unique_words) / len(words)

            # Cultural authenticity
            if self._check_cultural_authenticity(name, profile.cuisine_type):
                score += 2
                analysis["cultural_authentic"] = True

            # SEO friendliness (no special characters, reasonable length)
            if re.match(r'^[a-zA-Z\s&\-\.\']+$', name) and length <= 30:
                score += 1
                analysis["seo_friendly"] = True

            # Emotional appeal scoring
            emotional_words = {
                "positive": ["bella", "golden", "royal", "fresh", "garden", "home", "family", "cozy", "warm"],
                "elegant": ["grand", "premier", "signature", "classic", "fine", "maison", "palace"],
                "modern": ["urban", "new", "fresh", "modern", "contemporary", "fusion"]
            }

            for category, words_list in emotional_words.items():
                if any(word in name.lower() for word in words_list):
                    score += 1
                    analysis[f"{category}_appeal"] = True
                    break

            # Final scoring
            final_score = min(10, max(1, score))

            scored_names.append({
                **name_data,
                "score": final_score,
                "analysis": analysis,
                "length": length,
                "word_count": len(words)
            })

        # Sort by score and return
        return sorted(scored_names, key=lambda x: x.get("score", 0), reverse=True)

    def _check_cultural_authenticity(self, name: str, cuisine: str) -> bool:
        """Check if name has appropriate cultural elements"""
        cultural_indicators = {
            "Italian": ["bella", "casa", "nonna", "villa", "tavola", "amore", "sole", "luna"],
            "French": ["chez", "maison", "petit", "grand", "bon", "belle", "rouge", "noir"],
            "Japanese": ["sakura", "yuki", "hana", "kaze", "mizu", "tsuki", "aki", "natsu"],
            "Mexican": ["casa", "cantina", "sol", "luna", "corazon", "fiesta", "pueblo"],
            "Indian": ["taj", "maharaja", "palace", "garden", "spice", "masala", "curry"],
            "Chinese": ["golden", "dragon", "phoenix", "jade", "pearl", "bamboo", "lotus"]
        }

        if cuisine in cultural_indicators:
            return any(indicator in name.lower() for indicator in cultural_indicators[cuisine])
        return False

    def _generate_fallback_names(self, profile: RestaurantProfile, count: int) -> List[Dict]:
        """High-quality fallback names"""
        templates = [
            f"The {profile.cuisine_type} Corner",
            f"{profile.location} {profile.style}",
            f"Golden {profile.cuisine_type} Kitchen",
            f"Bella {profile.cuisine_type} House",
            f"Urban {profile.cuisine_type} Co.",
            f"Fresh {profile.cuisine_type} Works",
            f"Classic {profile.cuisine_type} Bistro",
            f"Modern {profile.cuisine_type} Lab"
        ]

        return [{"name": template, "score": 6, "reasoning": "Fallback generation", "source": "fallback"}
                for template in templates[:count]]

    def _store_generation(self, profile: RestaurantProfile, names: List[Dict]):
        """Store generation data for analytics"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        session_id = hashlib.md5(f"{datetime.now()}{profile.cuisine_type}".encode()).hexdigest()

        cursor.execute('''
            INSERT INTO generations
            (cuisine_type, style, location, target_audience, budget_range, theme,
             atmosphere, unique_features, competitor_analysis, generated_names, session_id)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            profile.cuisine_type, profile.style, profile.location, profile.target_audience,
            profile.budget_range, profile.theme, profile.atmosphere, profile.unique_features,
            profile.competitor_analysis, json.dumps([n["name"] for n in names]), session_id
        ))

        conn.commit()
        conn.close()

    def get_analytics(self) -> Dict:
        """Generate comprehensive analytics"""
        conn = sqlite3.connect(self.db_path)

        # Basic stats
        total_generations = pd.read_sql_query("SELECT COUNT(*) as count FROM generations", conn).iloc[0]['count']

        # Popular cuisines
        cuisine_stats = pd.read_sql_query("""
            SELECT cuisine_type, COUNT(*) as count
            FROM generations
            GROUP BY cuisine_type
            ORDER BY count DESC
            LIMIT 10
        """, conn)

        # Popular styles
        style_stats = pd.read_sql_query("""
            SELECT style, COUNT(*) as count
            FROM generations
            GROUP BY style
            ORDER BY count DESC
            LIMIT 10
        """, conn)

        # Daily activity (last 30 days)
        daily_activity = pd.read_sql_query("""
            SELECT DATE(timestamp) as date, COUNT(*) as generations
            FROM generations
            WHERE timestamp >= date('now', '-30 days')
            GROUP BY DATE(timestamp)
            ORDER BY date
        """, conn)

        conn.close()

        return {
            "total_generations": total_generations,
            "cuisine_stats": cuisine_stats,
            "style_stats": style_stats,
            "daily_activity": daily_activity
        }

    def create_analytics_plots(self, analytics_data: Dict):
        """Create interactive analytics plots"""
        plots = {}

        # Cuisine popularity pie chart
        if not analytics_data["cuisine_stats"].empty:
            fig_cuisine = px.pie(
                analytics_data["cuisine_stats"],
                values='count',
                names='cuisine_type',
                title="Most Popular Cuisines"
            )
            plots["cuisine_chart"] = fig_cuisine

        # Style popularity bar chart
        if not analytics_data["style_stats"].empty:
            fig_style = px.bar(
                analytics_data["style_stats"],
                x='style',
                y='count',
                title="Restaurant Styles Popularity"
            )
            plots["style_chart"] = fig_style

        # Daily activity line chart
        if not analytics_data["daily_activity"].empty:
            fig_activity = px.line(
                analytics_data["daily_activity"],
                x='date',
                y='generations',
                title="Generation Activity (Last 30 Days)"
            )
            plots["activity_chart"] = fig_activity

        return plots

# Initialize the advanced generator
generator = AdvancedRestaurantGenerator("b6860179930a45928369eec466ce4b4f")

def generate_names_interface(cuisine_type, style, location, target_audience, budget_range,
                           theme, atmosphere, unique_features, competitor_analysis, name_count):
    """Advanced Gradio interface function"""
    try:
        profile = RestaurantProfile(
            cuisine_type=cuisine_type,
            style=style,
            location=location,
            target_audience=target_audience,
            budget_range=budget_range,
            theme=theme,
            atmosphere=atmosphere,
            unique_features=unique_features,
            competitor_analysis=competitor_analysis
        )

        result = generator.generate_advanced_names(profile, int(name_count))

        # Format output with detailed information
        output_lines = []
        for i, name_data in enumerate(result["names"], 1):
            score = name_data.get("score", 0)
            reasoning = name_data.get("reasoning", "")
            positioning = name_data.get("positioning", "")

            line = f"""
🏆 **{i}. {name_data['name']}** (Score: {score}/10)
   📍 Positioning: {positioning.title()}
   💡 Reasoning: {reasoning}
   📊 Analysis: Length: {name_data.get('length', 0)} chars, Words: {name_data.get('word_count', 0)}
"""
            output_lines.append(line)

        # Add generation statistics
        stats = result.get("generation_stats", {})
        stats_text = f"""
📈 **Generation Statistics:**
• Total Names Generated: {stats.get('total_generated', 0)}
• AI Generated: {stats.get('ai_generated', 0)}
• Algorithmic Generated: {stats.get('algorithmic_generated', 0)}
• Average Quality Score: {stats.get('average_score', 0):.1f}/10
"""

        return "\n".join(output_lines) + "\n" + stats_text

    except Exception as e:
        return f"❌ Error generating names: {str(e)}"

def get_analytics_interface():
    """Get analytics data for display"""
    try:
        analytics = generator.get_analytics()
        plots = generator.create_analytics_plots(analytics)

        summary = f"""
📊 **Analytics Summary:**
• Total Generations: {analytics['total_generations']}
• Most Popular Cuisine: {analytics['cuisine_stats'].iloc[0]['cuisine_type'] if not analytics['cuisine_stats'].empty else 'N/A'}
• Most Popular Style: {analytics['style_stats'].iloc[0]['style'] if not analytics['style_stats'].empty else 'N/A'}
"""

        return summary, plots.get("cuisine_chart"), plots.get("style_chart"), plots.get("activity_chart")

    except Exception as e:
        return f"Error loading analytics: {str(e)}", None, None, None

# Create the advanced Gradio interface
def create_advanced_interface():
    with gr.Blocks(title="Advanced Restaurant Name Generator", theme=gr.themes.Soft()) as demo:
        gr.Markdown(
            """
            # 🍽️ Advanced AI Restaurant Name Generator
            ### Professional-grade restaurant naming with comprehensive analysis and analytics

            Generate creative, market-ready restaurant names using advanced AI and linguistic analysis.
            Each name comes with detailed scoring, positioning analysis, and cultural authenticity checks.
            """
        )

        with gr.Tabs():
            # Main Generator Tab
            with gr.TabItem("🎯 Name Generator"):
                with gr.Row():
                    with gr.Column(scale=1):
                        gr.Markdown("### Restaurant Profile")

                        cuisine_dropdown = gr.Dropdown(
                            choices=[
                                "Italian", "French", "Chinese", "Japanese", "Mexican", "Indian",
                                "Thai", "Greek", "Spanish", "Korean", "Vietnamese", "Mediterranean",
                                "American", "German", "British", "Brazilian", "Lebanese", "Turkish",
                                "Ethiopian", "Moroccan", "Fusion", "International"
                            ],
                            label="🍜 Cuisine Type",
                            value="Italian"
                        )

                        style_dropdown = gr.Dropdown(
                            choices=[
                                "Fine Dining", "Casual Dining", "Fast Casual", "Quick Service",
                                "Bistro", "Cafe", "Food Truck", "Family Restaurant", "Sports Bar",
                                "Wine Bar", "Cocktail Lounge", "Buffet", "Street Food", "Pop-up",
                                "Ghost Kitchen", "Catering", "Bakery Cafe", "Deli"
                            ],
                            label="🏪 Restaurant Style",
                            value="Casual Dining"
                        )

                        location_input = gr.Textbox(
                            label="📍 Location & Context",
                            placeholder="e.g., Downtown Seattle, Shopping Mall, Beach Boardwalk, University District",
                            lines=2
                        )

                        target_audience = gr.Dropdown(
                            choices=[
                                "Families with Children", "Young Professionals", "College Students",
                                "Business Executives", "Tourists", "Local Community", "Food Enthusiasts",
                                "Health-Conscious Diners", "Budget-Conscious", "Luxury Seekers",
                                "Date Night Couples", "Large Groups", "Solo Diners"
                            ],
                            label="👥 Target Audience",
                            value="Families with Children"
                        )

                        budget_range = gr.Dropdown(
                            choices=["$ (Under $15)", "$$ ($15-30)", "$$$ ($30-60)", "$$$$ ($60+)"],
                            label="💰 Price Range",
                            value="$$ ($15-30)"
                        )

                    with gr.Column(scale=1):
                        gr.Markdown("### Brand Identity")

                        theme_input = gr.Textbox(
                            label="🎨 Theme & Concept",
                            placeholder="e.g., Rustic farmhouse, Modern minimalist, Vintage retro, Industrial chic",
                            lines=2
                        )

                        atmosphere_input = gr.Textbox(
                            label="🌟 Desired Atmosphere",
                            placeholder="e.g., Cozy and intimate, Energetic and vibrant, Sophisticated and elegant",
                            lines=2
                        )

                        unique_features = gr.Textbox(
                            label="✨ Unique Features",
                            placeholder="e.g., Open kitchen, Live music, Outdoor seating, Farm-to-table, Craft cocktails",
                            lines=2
                        )

                        competitor_analysis = gr.Textbox(
                            label="🏢 Competitor Analysis",
                            placeholder="e.g., Need to stand out from Olive Garden and local Italian places, avoid names like...",
                            lines=3
                        )

                        name_count = gr.Slider(
                            minimum=5,
                            maximum=15,
                            value=8,
                            step=1,
                            label="📊 Number of Names to Generate"
                        )

                        generate_btn = gr.Button("🚀 Generate Advanced Names", variant="primary", size="lg")

                with gr.Row():
                    output = gr.Textbox(
                        label="🎯 Generated Restaurant Names with Analysis",
                        lines=25,
                        placeholder="Your professionally analyzed restaurant names will appear here...",
                        show_copy_button=True
                    )

                generate_btn.click(
                    fn=generate_names_interface,
                    inputs=[
                        cuisine_dropdown, style_dropdown, location_input, target_audience,
                        budget_range, theme_input, atmosphere_input, unique_features,
                        competitor_analysis, name_count
                    ],
                    outputs=output
                )

            # Analytics Tab
            with gr.TabItem("📈 Analytics Dashboard"):
                gr.Markdown("### Generation Analytics & Insights")

                analytics_btn = gr.Button("🔄 Refresh Analytics", variant="secondary")

                with gr.Row():
                    analytics_summary = gr.Textbox(
                        label="📊 Summary Statistics",
                        lines=5,
                        interactive=False
                    )

                with gr.Row():
                    with gr.Column():
                        cuisine_chart = gr.Plot(label="🍜 Popular Cuisines")
                        activity_chart = gr.Plot(label="📅 Generation Activity")

                    with gr.Column():
                        style_chart = gr.Plot(label="🏪 Popular Styles")

            analytics_btn.click(
                fn=get_analytics_interface,
                outputs=[analytics_summary, cuisine_chart, style_chart, activity_chart]
            )

            # Help Tab
            with gr.TabItem("❓ Help & Tips"):
                gr.Markdown("""
                ### 🎯 How to Get the Best Results

                #### 📝 Fill Out Complete Profiles
                - **Be Specific**: The more detailed your inputs, the better the AI can understand your vision
                - **Think Like a Customer**: Consider what would attract your target audience
                - **Research Competitors**: Mention existing restaurants to avoid similar names

                #### 🎨 Naming Strategy Tips
                - **Cultural Authenticity**: For ethnic cuisines, consider authentic language elements
                - **Location Relevance**: Include local landmarks or neighborhood characteristics
                - **Memorability**: Shorter names (8-20 characters) are often more memorable
                - **Pronunciation**: Ensure your target audience can easily say the name

                #### 📊 Understanding the Scoring System
                - **Score 9-10**: Exceptional names ready for immediate use
                - **Score 7-8**: Strong names that may need minor refinement
                - **Score 5-6**: Good starting points for further development
                - **Score 1-4**: Names needing significant improvement

                #### 🚀 Advanced Features
                - **Linguistic Analysis**: Each name is analyzed for pronounceability, memorability, and cultural fit
                - **Market Positioning**: Names are categorized by their market appeal
                - **SEO Optimization**: Names are checked for online search friendliness
                - **Analytics Tracking**: View trends and popular choices over time

                #### 💡 Industry Best Practices
                - **Trademark Research**: Always verify availability before finalizing
                - **Domain Availability**: Check if .com domains are available
                - **Social Media Handles**: Verify consistent handles across platforms
                - **Local Regulations**: Some areas have naming restrictions
                - **Focus Groups**: Test top names with your target demographic
                """)

        # Load initial analytics on startup
        demo.load(
            fn=get_analytics_interface,
            outputs=[analytics_summary, cuisine_chart, style_chart, activity_chart]
        )

    return demo

# Enhanced batch generation function
def batch_generate_names(profiles_csv_path: str) -> str:
    """Generate names for multiple restaurants from CSV input"""
    try:
        df = pd.read_csv(profiles_csv_path)
        results = []

        for _, row in df.iterrows():
            profile = RestaurantProfile(
                cuisine_type=row.get('cuisine_type', 'Italian'),
                style=row.get('style', 'Casual Dining'),
                location=row.get('location', ''),
                target_audience=row.get('target_audience', 'Families'),
                budget_range=row.get('budget_range', '$'),
                theme=row.get('theme', ''),
                atmosphere=row.get('atmosphere', ''),
                unique_features=row.get('unique_features', ''),
                competitor_analysis=row.get('competitor_analysis', '')
            )

            result = generator.generate_advanced_names(profile, 5)
            top_names = [name['name'] for name in result['names'][:3]]

            results.append({
                'input_cuisine': profile.cuisine_type,
                'input_style': profile.style,
                'generated_name_1': top_names[0] if len(top_names) > 0 else '',
                'generated_name_2': top_names[1] if len(top_names) > 1 else '',
                'generated_name_3': top_names[2] if len(top_names) > 2 else '',
                'generation_timestamp': datetime.now().isoformat()
            })

        # Save results
        results_df = pd.DataFrame(results)
        output_path = f"batch_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        results_df.to_csv(output_path, index=False)

        return f"Batch generation complete! Results saved to {output_path}"

    except Exception as e:
        return f"Batch generation error: {str(e)}"

# Advanced API endpoint simulation
class RestaurantNameAPI:
    def __init__(self, generator):
        self.generator = generator

    def generate_via_api(self, request_data: Dict) -> Dict:
        """Simulate API endpoint for name generation"""
        try:
            profile = RestaurantProfile(**request_data)
            result = self.generator.generate_advanced_names(profile)

            return {
                "status": "success",
                "data": {
                    "names": result["names"],
                    "statistics": result.get("generation_stats", {}),
                    "generation_id": hashlib.md5(str(datetime.now()).encode()).hexdigest(),
                    "timestamp": datetime.now().isoformat()
                }
            }
        except Exception as e:
            return {
                "status": "error",
                "message": str(e),
                "timestamp": datetime.now().isoformat()
            }

# Initialize API handler
api_handler = RestaurantNameAPI(generator)

# Enhanced export functionality
def export_generation_history() -> str:
    """Export all generation history as CSV"""
    try:
        conn = sqlite3.connect(generator.db_path)
        df = pd.read_sql_query("""
            SELECT
                timestamp,
                cuisine_type,
                style,
                location,
                target_audience,
                budget_range,
                theme,
                atmosphere,
                generated_names,
                user_rating
            FROM generations
            ORDER BY timestamp DESC
        """, conn)
        conn.close()

        export_path = f"generation_history_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        df.to_csv(export_path, index=False)

        return f"History exported to {export_path}"

    except Exception as e:
        return f"Export error: {str(e)}"

# Performance monitoring
class PerformanceMonitor:
    def __init__(self):
        self.metrics = {
            "total_requests": 0,
            "successful_generations": 0,
            "failed_generations": 0,
            "average_response_time": 0,
            "api_calls_made": 0,
            "cache_hits": 0
        }
        self.response_times = []

    def log_request(self, success: bool, response_time: float, used_cache: bool = False):
        self.metrics["total_requests"] += 1

        if success:
            self.metrics["successful_generations"] += 1
        else:
            self.metrics["failed_generations"] += 1

        if used_cache:
            self.metrics["cache_hits"] += 1
        else:
            self.metrics["api_calls_made"] += 1

        self.response_times.append(response_time)
        self.metrics["average_response_time"] = sum(self.response_times) / len(self.response_times)

    def get_performance_report(self) -> Dict:
        success_rate = (self.metrics["successful_generations"] / self.metrics["total_requests"] * 100) if self.metrics["total_requests"] > 0 else 0

        return {
            **self.metrics,
            "success_rate_percentage": round(success_rate, 2),
            "cache_hit_rate": round((self.metrics["cache_hits"] / self.metrics["total_requests"] * 100) if self.metrics["total_requests"] > 0 else 0, 2)
        }

# Initialize performance monitor
performance_monitor = PerformanceMonitor()

# Enhanced caching system
class NameCache:
    def __init__(self, max_size: int = 1000):
        self.cache = {}
        self.access_times = {}
        self.max_size = max_size

    def _generate_key(self, profile: RestaurantProfile) -> str:
        """Generate cache key from profile"""
        key_data = f"{profile.cuisine_type}{profile.style}{profile.location}{profile.target_audience}"
        return hashlib.md5(key_data.encode()).hexdigest()

    def get(self, profile: RestaurantProfile) -> Optional[Dict]:
        """Get cached result if available"""
        key = self._generate_key(profile)
        if key in self.cache:
            self.access_times[key] = time.time()
            return self.cache[key]
        return None

    def set(self, profile: RestaurantProfile, result: Dict):
        """Cache generation result"""
        if len(self.cache) >= self.max_size:
            # Remove least recently used item
            oldest_key = min(self.access_times.keys(), key=lambda k: self.access_times[k])
            del self.cache[oldest_key]
            del self.access_times[oldest_key]

        key = self._generate_key(profile)
        self.cache[key] = result
        self.access_times[key] = time.time()

# Initialize cache
name_cache = NameCache()

# Enhanced generation with caching and monitoring
def enhanced_generate_names(profile: RestaurantProfile, count: int = 8) -> Dict:
    """Enhanced generation with caching and performance monitoring"""
    start_time = time.time()

    try:
        # Check cache first
        cached_result = name_cache.get(profile)
        if cached_result:
            response_time = time.time() - start_time
            performance_monitor.log_request(True, response_time, used_cache=True)
            return cached_result

        # Generate new names
        result = generator.generate_advanced_names(profile, count)

        # Cache the result
        name_cache.set(profile, result)

        # Log performance
        response_time = time.time() - start_time
        performance_monitor.log_request(True, response_time, used_cache=False)

        return result

    except Exception as e:
        response_time = time.time() - start_time
        performance_monitor.log_request(False, response_time)
        raise e

# Launch the application with enhanced features
if __name__ == "__main__":
    print("🚀 Starting Advanced Restaurant Name Generator...")
    print("📊 Initializing database and analytics...")
    print("🔧 Setting up caching and performance monitoring...")

    demo = create_advanced_interface()

    print("✅ All systems ready!")
    print("🌐 Launching web interface...")

    demo.launch(
        share=True,  # Creates public link for Steam
        server_name="0.0.0.0",
        server_port=7860,
        debug=True,
        show_error=True,
        favicon_path=None,
        ssl_verify=False,
        quiet=False
    )

    print("🎉 Advanced Restaurant Name Generator is now live!")
    print("📈 Analytics dashboard available in the Analytics tab")
    print("💡 Check the Help tab for best practices and tips")

🚀 Starting Advanced Restaurant Name Generator...
📊 Initializing database and analytics...
🔧 Setting up caching and performance monitoring...
✅ All systems ready!
🌐 Launching web interface...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://977a0e2b3b13b5c18a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Generation error: API call failed: 405
Generation error: API call failed: 405
Generation error: API call failed: 405
Keyboard interruption in main thread... closing server.
Killing tunnel 0.0.0.0:7860 <> https://977a0e2b3b13b5c18a.gradio.live
🎉 Advanced Restaurant Name Generator is now live!
📈 Analytics dashboard available in the Analytics tab
💡 Check the Help tab for best practices and tips


In [1]:
!pip install gradio requests pandas plotly sqlite3 hashlib

ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3
